In [1]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from datetime import datetime

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)


In [2]:
acceso = open('../../api_key/api.key','r')
api_token = acceso.readlines()[0]
acceso.close()

amigocloud = AmigoCloud(token=api_token)
amigocloud

In [13]:
id_insp = 10

In [18]:
# DATOS DE LAS INPECCION
query = {'query': 'select * from dataset_315614 where id={id_insp}'.format(id_insp=id_insp)}
project = 'https://app.amigocloud.com/api/v1/projects/32920/sql'
select = amigocloud.get(project, query)
data = select['data'][0]

# convertir fecha a formato dd/mm/aaaa
data['fecha_de_registro'] = datetime.strptime(data['fecha_de_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')

# los campos booleanos tiene formacto 'num_num', esto no es admitido para convertir a objeto
# se modifica el dict para agregar 'k_' al inicio de todas llaves
data = {'k_'+key : value for (key, value) in data.items()}
insp = collections.namedtuple("insp", data.keys())(*data.values())
print('end')

end


In [19]:
insp

insp(k_fecha_de_registro='30/03/2023', k_amigo_id='226304fcb8e748ad94b89c4c8fcc441e', k_ubicacion='0101000020E61000008E7406465EA24FC0B62DCA6C904331C0', k_id=10, k_canhero='594 / AGROPECUARIA FLAMBOYAN', k_inspeccion='1RA', k_maquina='JOHN DEERE', k_modelo='3520', k_codigo_maquina=None, k_observaciones='Lá cosechadora 09 tá está 100% terminada fie realizado todos Los test lo cual está apta para cosechar ', k_1_1=False, k_1_2=False, k_1_3=False, k_1_4=False, k_1_5=False, k_2_1=False, k_2_2=False, k_2_3=False, k_2_4=False, k_3_1=False, k_3_2=False, k_3_3=False, k_3_4=False, k_3_5=False, k_3_6=False, k_3_7=False, k_3_8=False, k_4_1=False, k_4_2=False, k_4_3=False, k_4_4=False, k_4_5=False, k_5_1=False, k_5_2=False, k_5_3=False, k_5_4=False, k_5_5=False, k_5_6=False, k_6_1=False, k_6_2=False, k_6_3=False, k_6_4=False, k_6_5=False, k_7_1=False, k_7_2=False, k_7_3=False, k_7_4=False, k_7_5=False, k_7_6=False, k_7_7=False, k_7_8=False, k_8_1=False, k_8_2=False, k_8_3=False, k_8_4=False, k_8_5=

In [20]:
insp.k_amigo_id

'226304fcb8e748ad94b89c4c8fcc441e'

In [83]:
# FOTOS DE INSPECCION
query_fotos = {'query': 'select amigo_id, id, obs from dataset_315689 where insp_ref_fot = \'{id}\''.format(id=insp.k_amigo_id)}
project = 'https://app.amigocloud.com/api/v1/projects/32920/sql'
fotos = amigocloud.get(project, query_fotos)['data']

In [84]:
fotos

[{'amigo_id': '8e046dea89984cd186456db49b18b3eb',
  'id': 4,
  'obs': 'Se está empezando a realizar el mantenimiento de está maquina por ende se a dado un plazo de 15 dia para terminar su reparo y manutencion '},
 {'amigo_id': '09b5ae322144429e97918d344d84c08f', 'id': 5, 'obs': 'Obs 2'},
 {'amigo_id': 'da013f6d742948f88539277ec62028fe', 'id': 6, 'obs': 'Obs 3'}]

In [88]:
fotos_dict = []
doc = DocxTemplate("templates/01_template_operador.docx")
for i in fotos:
    
    query_f = {'query': 'select gal.s3_filename from dataset_315689 fot\
                            inner join gallery_48937 gal on fot.amigo_id=gal.source_amigo_id\
                            where fot.amigo_id=\'{id}\''.format(id=i['amigo_id'])}
    project = 'https://app.amigocloud.com/api/v1/projects/32920/sql'
    result = amigocloud.get(project, query_f)['data']
    
    
    lista_fotos_inline = []
    for foto in result:
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
        file = open('fotos/' + foto['s3_filename'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor='fotos/' + foto['s3_filename'], width=Mm(50))})
    
    i['fotos'] = lista_fotos_inline
    
    
    print(i)
    print('========================')
    print(result)

{'amigo_id': '8e046dea89984cd186456db49b18b3eb', 'id': 4, 'obs': 'Se está empezando a realizar el mantenimiento de está maquina por ende se a dado un plazo de 15 dia para terminar su reparo y manutencion ', 'fotos': [{'foto': <docxtpl.inline_image.InlineImage object at 0x0000016461A80388>}, {'foto': <docxtpl.inline_image.InlineImage object at 0x0000016461A8C148>}, {'foto': <docxtpl.inline_image.InlineImage object at 0x0000016461765448>}]}
[{'s3_filename': 'IMG_20230328_150113.jpg'}, {'s3_filename': 'IMG_20230328_150133.jpg'}, {'s3_filename': 'IMG_20230328_150142.jpg'}]
{'amigo_id': '09b5ae322144429e97918d344d84c08f', 'id': 5, 'obs': 'Obs 2', 'fotos': [{'foto': <docxtpl.inline_image.InlineImage object at 0x0000016461765748>}]}
[{'s3_filename': '48937_20230329031258762.jpg'}]
{'amigo_id': 'da013f6d742948f88539277ec62028fe', 'id': 6, 'obs': 'Obs 3', 'fotos': [{'foto': <docxtpl.inline_image.InlineImage object at 0x00000164617AAE88>}]}
[{'s3_filename': '48937_20230329031317537.jpg'}]


In [23]:
doc = DocxTemplate("templates/template_insp_cosechadoras_pre.docx")

#descargar fotos y generar lista InlineImage
'''
lista_fotos_inline = []
for foto in fotos:
    
    response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
    file = open('fotos/' + foto['s3_filename'], "wb")
    file.write(response.content)
    file.close()
    lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor='fotos/' + foto['s3_filename'], width=Mm(50))})
'''

#context = {'insp':insp, 'fotos':lista_fotos_inline}
context = {'insp':insp}

doc.render(context)

# formato de nombre de archivo: "123_CQ_01-01-2022_NOMBRE"
cod_nom = insp.k_canhero.split(' / ')
file_name = cod_nom[0] + '_ECP_' + insp.k_fecha_de_registro.replace('/','-') + '_' + cod_nom[1] + '_' + str(insp.k_id)

#doc.save(r'C:\Users\UTEA\OneDrive - Ingenio Azucarero Guabira S.A (2)\INFORMES\IMFORMES PARA REVISAR\\' + file_name + '.docx')
doc.save(r'_' + file_name + '.docx')